In [77]:
import numpy as np

In [78]:
import mne

In [79]:
import pandas as pd

In [80]:
import json

In [81]:
%matplotlib
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [82]:
%load_ext autoreload

%autoreload 2
import tools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
import numpy as np

In [84]:
import itertools

In [85]:
EMPTY_ACTION = -1

In [86]:
import itertools

In [87]:
import eeg_events

In [88]:
class OfflineJoluEngagementEnvironment():
    filename = None
    iterator = None
    event = None
    
    states = ["".join(t) for t in itertools.product(*[[str(i) for i in range(0, 6)]] * 3)]
    actions = {
        0: "o",
        1: "x",
        2: "n",
        3: "*",
        4: "u",
        5: "-",
        -1: "$"
    }
    
    def __init__(self, filename):
        self.filename = filename
    
    def reset(self):
        self.iterator = eeg_events.iterate_events(self.filename)
        self.event = next(self.iterator)
        return self.states[0]
    
    def sample(self):
        event = self.event[1]
        if not event.get("collect"):
            return self.actions[event["response"]['result']["reels"]["view"][0][0]]
        else:
            return self.actions[EMPTY_ACTION]
        
    def get_state(self):
        event_tuple = self.event
        event = event_tuple[1]
        state_array = event["response"]['result']["state"]["towerStages"]
        state = "".join([str(i) for i in state_array])
        return state
    
    def get_reward(self):
        event_tuple = self.event
        event_data = event_tuple[2]
        assert event_data.shape[0] > 0
        return np.median(event_data)
        
    def step(self, action):
        event = self.event
        
        next_state = self.get_state()
        reward = self.get_reward()
        
        self.event = next(self.iterator, None)
        
        done = not self.event
        
        return next_state, reward, done, None

In [90]:
env = OfflineJoluEngagementEnvironment('test-filtered')

In [91]:
env.reset()

Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.


'000'

In [92]:
done = False
while not done:
    print(env.sample())
    state_tuple = env.step('')
    _, _, done, _ = state_tuple
    print(state_tuple)

u
('001', 2.536606013774872, False, None)
u
('002', 2.535471558570862, False, None)
o
('113', 2.556123435497284, False, None)
$
('000', 2.8530492782592773, False, None)
-
('000', 2.975451350212097, False, None)
-
('000', 2.5551390647888184, False, None)
u
('001', 2.482528030872345, False, None)
-
('001', 2.7996336817741394, False, None)
-
('001', 3.506598949432373, False, None)
-
('001', 2.9627907276153564, False, None)
-
('001', 2.152766168117523, False, None)
-
('001', 2.2887578904628754, False, None)
*
('011', 2.429207146167755, False, None)
-
('011', 2.859145998954773, False, None)
*
('021', 2.7538114190101624, False, None)
$
('000', 2.506966769695282, False, None)
u
('001', 2.624290108680725, False, None)
-
('001', 2.5965721011161804, False, None)
-
('001', 2.3610068261623383, False, None)
*
('011', 2.4811218976974487, False, None)
-
('011', 2.5712570548057556, False, None)
-
('011', 2.6813647150993347, False, None)
-
('011', 2.7888752222061157, False, None)
*
('021', 2.8716769218

('002', 2.9064472317695618, False, None)
*
('012', 3.4807721376419067, False, None)
-
('012', 3.3296441435813904, False, None)
-
('012', 3.194740653038025, False, None)
-
('012', 3.5442144870758057, False, None)
-
('012', 3.5145362615585327, False, None)
u
('013', 3.2618125081062317, False, None)
-
('013', 3.1286707520484924, False, None)
u
('014', 3.236755907535553, False, None)
$
('000', 3.249279201030731, False, None)
u
('001', 3.138131618499756, False, None)
-
('001', 3.0862314105033875, False, None)
u
('002', 2.5617677569389343, False, None)
-
('002', 2.5116496086120605, False, None)
x
('000', 2.577990233898163, False, None)
-
('000', 2.538322329521179, False, None)
u
('001', 2.178978145122528, False, None)
-
('001', 2.257950186729431, False, None)
-
('001', 2.537293791770935, False, None)
u
('002', 2.86731094121933, False, None)
-
('002', 3.313132882118225, False, None)
u
('003', 3.1322333216667175, False, None)
$
('000', 2.1242023706436157, False, None)
-
('000', 2.1943131983280

### SARSA

In [93]:
env = OfflineJoluEngagementEnvironment('test-filtered')

In [134]:
alpha = 0.15
gamma = 0.99

In [135]:
Q = {}

In [136]:
states_number = {}

In [137]:
from tqdm import tqdm_notebook

In [138]:
for i in tqdm_notebook(range(0, 100)):
    r = 0
    state = env.reset()
    action = env.sample()
    while True:
        nextstate, reward, done, _ = env.step(action)
        if done:
            break
        nextaction = env.sample()
#         print(state, action, nextstate, nextaction)
        if (state, action) not in Q:
            Q[(state, action)] = 0
            states_number[(state, action)] = []
        if (nextstate, nextaction) not in Q:
            Q[(nextstate, nextaction)] = 0
            states_number[(nextstate, nextaction)] = []
        Q[(state, action)] += alpha * (reward + gamma * Q[(nextstate, nextaction)] - Q[(state, action)])
        states_number[(state, action)].append(reward)
        action = nextaction
        state = nextstate
        r += reward

/home/anatoli/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw data fil

Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw d

Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw data file /home/anatoli/Документы/skywind/eeg-playground/sasha-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 98560 =      0.000 ...   385.000 secs
Ready.
Opening raw d

In [139]:
symbols_engagement = list(filter(lambda x: x[-1] != 0, sorted(Q.items(), key=lambda x: x[1])))

In [140]:
def calculate_reward(state):
    bells = int(state[0])
    stars = int(state[1])
    horsehoes = int(state[2])
    return joker_rewards["bell"][bells] + joker_rewards["star"][stars] + joker_rewards["horsehoe"][horsehoes]

In [141]:
joker_rewards = {
    "bell": [0, 10, 40, 100, 200, 350],
    "star": [0, 3, 8, 16, 29, 50],
    "horsehoe": [0, 1, 2, 4, 7, 10],
    "biggest": 500
}

In [142]:
symbols_engagement = [(a, calculate_reward(a), len(states_number[(a, b)]), b, c) for (a, b), c in symbols_engagement]

In [143]:
sorted(symbols_engagement, key=lambda x: x[-1])

[('000', 0, 100, 'n', 162.85223884506416),
 ('001', 1, 100, 'n', 163.82853577910726),
 ('001', 1, 100, 'o', 168.27296455401034),
 ('002', 2, 100, 'o', 169.41257650928958),
 ('010', 3, 100, '*', 169.96809705509574),
 ('101', 11, 100, '-', 170.07292669189818),
 ('100', 10, 100, 'u', 170.30704852933533),
 ('003', 4, 300, '-', 174.40032373110736),
 ('002', 2, 500, 'u', 174.7198258274229),
 ('112', 15, 100, '$', 175.7055705740514),
 ('003', 4, 100, '*', 176.61710953937896),
 ('003', 4, 100, '$', 176.75962303437527),
 ('113', 17, 100, '$', 176.824418182757),
 ('101', 11, 100, 'u', 176.8561859061828),
 ('020', 8, 100, 'u', 176.92694514299288),
 ('003', 4, 200, 'u', 177.22792176368546),
 ('101', 11, 100, '$', 177.83608492926294),
 ('000', 0, 100, 'x', 177.96132087538976),
 ('013', 7, 400, '-', 178.08432197748604),
 ('021', 9, 100, 'u', 178.61090943336197),
 ('002', 2, 200, 'n', 179.04698204159635),
 ('013', 7, 200, 'u', 179.18742255183454),
 ('102', 12, 200, '-', 179.5011092512925),
 ('013', 7

Пример - состояние 013. Вовлеченность существенно больше, если игрок находится в состоянии 0. Что это за состояние?

Как меняются приоритеты игрока в состояниях 0 и 2

In [144]:
def increase_state(s, i):
    return s[:i] + chr(ord(s[i]) + 1) + s[i+1:]

In [145]:
def get_transition(s, a):
    if a == '-':
        return s
    if a in ['$', 'x']:
        return '000'
    if a == 'u':
        return increase_state(s, 2)
    if a == 'n':
        return increase_state(s, 0)
    if a == '*':
        return increase_state(s, 1)
    if a == 'o':
        ns = increase_state(s, 0)
        ns = increase_state(ns, 1)
        ns = increase_state(ns, 2)
        return ns

In [146]:
get_transition("000", "o")

'111'

In [147]:
rewards = {}

In [148]:
for key, value in list(Q.items()):
    if not value:
        del Q[key]

In [149]:
V = {} # V - minimum value
for (s, a), value in Q.items():
    if s not in V:
        V[s] = max(Q.values())
    if value < V[s]:
        V[s] = value

In [150]:
sorted(V.items(), key=lambda x: x[1])

[('000', 162.85223884506416),
 ('001', 163.82853577910726),
 ('002', 169.41257650928958),
 ('010', 169.96809705509574),
 ('101', 170.07292669189818),
 ('100', 170.30704852933533),
 ('003', 174.40032373110736),
 ('112', 175.7055705740514),
 ('113', 176.824418182757),
 ('020', 176.92694514299288),
 ('013', 178.08432197748604),
 ('021', 178.61090943336197),
 ('102', 179.5011092512925),
 ('012', 179.70395959708821),
 ('004', 180.50143219271212),
 ('011', 181.33979087492622),
 ('014', 181.67476361843714),
 ('022', 184.42191008467285)]

In [151]:
for (s, a), value in Q.items():
    ns = get_transition(s, a)
    if (s, a) not in rewards:
        rewards[(s, a)] = 0
    rewards[(s, a)] = Q[(s, a)] - V[ns] / gamma

In [152]:
sorted(rewards.items(), key=lambda x: x[1])

[(('001', 'o'), -9.207409763213292),
 (('002', 'o'), -9.197946907636691),
 (('000', 'n'), -9.175082901739216),
 (('010', '*'), -8.745988947927373),
 (('001', 'n'), -7.962299263214135),
 (('021', 'u'), -7.673848227923742),
 (('003', 'u'), -5.096757218852048),
 (('101', 'u'), -4.458065862799515),
 (('013', 'u'), -4.322439689011048),
 (('020', 'u'), -3.488114890706072),
 (('003', '*'), -3.266043973233195),
 (('012', '*'), -2.5307780889380638),
 (('002', 'n'), -2.2672697273859512),
 (('102', '-'), -1.813142517689812),
 (('013', '-'), -1.7988315351261122),
 (('003', '-'), -1.761619431627338),
 (('101', '-'), -1.7179083504232153),
 (('100', 'u'), -1.4837865129860575),
 (('002', 'u'), -1.4421173353117922),
 (('012', 'u'), 0.019763386682711825),
 (('001', '*'), 0.4625647696471731),
 (('011', '-'), 0.6479089120611263),
 (('011', '*'), 0.924730841227273),
 (('012', '-'), 1.6524872611813635),
 (('010', 'u'), 1.8852386641044632),
 (('011', 'u'), 1.9585108643396438),
 (('002', '*'), 3.4959218620807